In [ ]:
library(tidyverse)
library(hrbrthemes)
library(zoo)
options(repr.plot.width = 18, repr.plot.height = 18)

In [ ]:
#read demograpgics file, QualityCheck file, and Volumes file

In [ ]:
demog <- 
qc <- 
vols <- 

In [ ]:
vols %>%
filter( ids == "sub-k10010")

In [ ]:
#normalization 

vols <- vols %>%   mutate(
    across(
      c('Frontal', 'Temporal', 'Hippocampus', 'Frontal_Parietal', 
        'Parietal', 'Insula', 'Cingulate', 'Occipital', 'BasalGanglia', 
        'Thalamus', 'Cerebellum', 'CorpusCallosum', 'Ventricles', 
        'VentralDC', 'WM', 'Brainstem'),
      ~ . / (eTIV)
    )
  )


In [ ]:
vols <- merge(demog,vols, by='ids')

In [ ]:
data <- merge(vols,
              qc,
              by='ids') %>% filter(inclusion == 'yes') %>% select(-inclusion)

In [ ]:
data %>% head()

In [ ]:

mov_perc <- data %>% 
select (-sex) %>%
gather(key='ROI', value='Volume', -c(ids, age, eTIV, hemisphere,)) %>%
mutate(r_age = round(age,0)) %>%
arrange(age) %>%
group_by(ROI) %>%
summarize(p05=rollapply(Volume,5,quantile,prob=0.05, fill=NA),
          p50=rollapply(Volume,5,quantile,prob=0.50,fill=NA),
         p95=rollapply(Volume,5,quantile,prob=0.95,fill=NA))

mov_perc %>% head()

In [ ]:
data %>% 
gather(key='ROI', value='Volume', -c(ids, age, sex, hemisphere, eTIV)) %>%
arrange(ROI, age) %>%
mutate(p05 = mov_perc$p05, p50 = mov_perc$p50, p95 = mov_perc$p95) %>%

ggplot(aes(x=age, y=Volume, color=ROI))+
geom_jitter(aes(shape = hemisphere), alpha=0.3)+
geom_smooth(aes(x=age, y=p05), color='black', se=F, method='loess', span = 1)+
geom_smooth(aes(x=age, y=p50), color='black', se=F, method='loess', span = 1)+
geom_smooth(aes(x=age, y=p95), color='black', se=F, method='loess', span = 1)+
facet_wrap(~ROI, scale='free_y')+
ylab("Relative Volume") + 
xlab("Age (year)")+
theme_ipsum()+
scale_x_continuous(breaks = c(seq(0, 18, by = 3))) + 


scale_color_viridis_d(option='A',begin =0 , end = 0.75) + # Specify darker shades
    theme(
    axis.title.x=element_text(size=45),
    axis.title.y=element_text(size=45),
    strip.text = element_text(size = 22, face = "bold"),
    axis.text.x = element_text(size=25),#, angle=90,hjust=0.95,vjust=0.2),
    axis.text.y = element_text(size=25),
    legend.title = element_text(size=22),
    legend.text = element_text(size=22),
            legend.position='none'

     )

In [ ]:
# Save data frames as RDS
#saveRDS(data, "Percetilcurves_LR.rds")

In [ ]:
#data %>% select(-c('ids','sex','age','eTIV', 'hemisphere')) %>% rowSums